In [ ]:
#import packages
import matplotlib.pyplot as plt
import numpy as np

## Steps for simulating continuum radio sky map:
### Step1:
    Goal: package a particular halo catalog into a dictionary that can easily get wanted message.
    Input: halo catalog file
    Return: a dictionary with keys including: location( in spherical coordinates), redshift, number, subhalo. halovolume, and so on.
    
    1.From halo catalog get location, velocity, number, subhalos and some other nesessary data.
    2.Cut the centrol boll out and transfer every location data into spherical coordinates.
    3.Transfer into z space using velocity information.
    4.Return a dictionary
### Step2:
    Goal: make a function that can simulate the galaxies with a particular number in each halo in a z slice and a L bin and a nu bin.
    AGN: 
    Input: nu(MHz), parameter_list, Luminocity, z, **(nu_bin, z_bin, L_bin)
    Return: numbers of galaxies in each halo with particular location data
    
    1.Make nesessary funcions( LF, Numbercounts, L(z), S(nu)).
    2.Count the number density using luminocity function
    3.Cut the space into some small solid angles, each solid angle related to a small volume. Count the total number for this small volume. For each solid angle, give each halo a number of galaxies that is related to its halo mass, and with a total galaxies number counted before.
    4.Give each halo a fluctuation of number counts, without changing the total number counts.
    5.For each solid angle, each redshift bin, do the same thing mentioned before. 
    6.Rerurn the new array that include number counts and locations.
    7.Draw the picture in healpix way.
### Step3:( can skip it at first)
    Goal:seperate the galaxies
    Input: none
    Return: every galaxies' location
    
    1.For every halo, make an array with length equal to the number of galaxies in it.
    2.Give them a random location within the halo volume.
    3.Do the same thing to each halo, append the array.
    4.Return the array
    5.Draw the picture.
### Step4:
    Goal:Add all luminocites in a total map
    Input:Luminocity_min, Luminocity_max, Luminocity_bin
    Return:All luminocity, all locations
    
    1.Count all luminocity bin using method mentioned before.
    2.Make a big array, to hold total luminocity information. That is to say, add all arrays into a whole array, each with a particular luminocity.
    3.Return this big array
    4.Draw picture. You can see a huge amount of points, each shows to a galaxy, with a particular luminocity.
### Step5:
    Goal:Make the final function, that input a nu can return a galaxy simulation array. And give some statistical information.
    Input: nu,*nu_bin
    Return: galaxy simulation array, picture, and so on
    
    1.Conclude code typed before in to a function
    2.Draw pictures and do statistic analysis and compare these with observertory data.
### Step6:
    Goal: Advanced correlation...
    

## Step1:package a particular halo catalog into a dictionary that can easily get wanted message.

In [1]:
def get_data(filename,size = 200):    
    '''
    get_data(filename,**other)\n
    other can include the size of box. size = 200\n

    1.From halo catalog get location, velocity, number, subhalos and some other nesessary data.\n
    2.Cut the centrol boll out and transfer every location data into spherical coordinates.\n
    3.Transfer into z space using velocity information.\n
    4.Return a dictionary\n
    '''
    halocatalog = np.loadtxt(filename)
    #make a dictionary to hold all information
    halo_dic = {}
    halo_keys = halocatalog[0]
    for i in range(len(halo_keys)):
        key = halo_keys[i]
        halo_dic[key] = halocatalog[1:,i]

    #center the coordinates
    halo_dic["Xc(6)"] = halo_dic["Xc(6)"] - size*1000*0.5
    halo_dic["Yc(6)"] = halo_dic["Yc(6)"] - size*1000*0.5
    halo_dic["Zc(6)"] = halo_dic["Zc(6)"] - size*1000*0.5
    
    #transfer into spherical coordinates
    def appendspherical_np(x,y,z):
        r = np.sqrt(x**2+y**2+z**2)
        theta = np.arctan2(np.sqrt(x**2+y**2),z)
        phi = np.arctan2(y,x)
        return r, theta, phi
    halo_dic["R"], halo_dic["Theta"], halo_dic["Phi"] = appendspherical_np(halo_dic['Xc(6)'], halo_dic['Yc(6)'], halo_dic['Zc(6)'])
    #transfer into red shift space
    H0 = 67.77
    c = 299792.458
    halo_dic['Z'] = halo_dic['R'] * H0 / c / 1000
    halo_dic_centrol = {}

    # cut the centrol ball out
    for i in halo_dic['#ID(1)']:
        if halo_dic["R"][i]<= size*1000/2:
            for key in halo_dic:
                halo_dic_centrol[key].append(halo_dic[key][i])
        else:
            pass
    return halo_dic_centrol

#get the data you want
def wanted(halo_dic, **content):
    ''' 
    content include: #ID(1)	hostHalo(2)	numSubStruct(3)	Mvir(4)	npart(5)	Xc(6)	Yc(7)\n
    Zc(8)	VXc(9)	VYc(10)	VZc(11)	Rvir(12)	Rmax(13)	r2(14)	mbp_offset(15)\n	
    com_offset(16)	Vmax(17)	v_esc(18)	sigV(19)	lambda(20)	lambdaE(21)\n	
    Lx(22)	Ly(23)	Lz(24)	b(25)	c(26)	Eax(27)	Eay(28)	Eaz(29)	Ebx(30)	Eby(31)\n	
    Ebz(32)	Ecx(33)	Ecy(34)	Ecz(35)	ovdens(36)	nbins(37)	fMhires(38)	Ekin(39)\n	
    Epot(40)	SurfP(41)	Phi0(42)	cNFW(43)    R    Theta    Phi    Z\n
    if you don't type anything, it will return #ID(1), Mvir(4), Theta, Phi and Z.
    '''
    halo_dic_want = {}
    halo_dic_want['#ID(1)'] = halo_dic['#ID(1)']
    halo_dic_want['Mvir(4)'] = halo_dic['Mvir(4)']
    halo_dic_want['Theta'] = halo_dic['Theta']
    halo_dic_want['Phi'] = halo_dic['Phi']
    halo_dic_want['Z'] = halo_dic['Z']
    for key in content:
        halo_dic_want[key] = halo_dic[key]
    return halo_dic_want



# Step2: make a function that can simulate the galaxies with a particular number in each halo in a z slice and a S bin and a nu bin.

In [1]:
#1.Make nesessary funcions( LF, Numbercounts, L(z), S(nu)).

def logL_0(z, logLz):
    ''' 
    asume that logL(z) has the same evolution method as logL_star(z)\n
    but change the L dependent z_top into L independent
    input logLz, return logL0 for farther calculating
    '''
    k_evo = para['k_evo']
    z_top0 = para['z_top0']
    dz_top = para['dz_top']
    m_ev = para['m_ev']
    z_top = z_top0 + dz_top
    
    logL0 = logLz - (k_evo*z*(2*z_top - z**m_ev*z_top**(1-m_ev)/(1+m_ev)))
    return logL0

def logL_Sz(S, z):
    ''' 
    S is the flux, with unit mJy(10^(-29)W/m^2Hz)\n
    z is redshift\n
    return logLz and logL0
    '''
    H0 = 67.77
    c = 299792.458
    logLz = np.log(4*np.pi*(z*c/H0)*(10**6*3.0836*10**16)**2*S*10**(-29))
    logL0 = logL_0(z, logLz)
    return logLz, logL0

def logL_z(logL0, z):
    ''' 
    to calculate the logL(z) if we already know logL(0)\n
    logL(0) is dependent on what we expect to see in redshift z\', but we want to know how it evolute in the past time.
    input logL0, z **para
    return logL_z
    '''
    k_evo = para['k_evo']
    z_top0 = para['z_top0']
    dz_top = para['dz_top']
    m_ev = para['m_ev']
    z_top = z_top0 + dz_top
    
    logLz = logL0 + (k_evo*z*(2*z_top - z**m_ev*z_top**(1-m_ev)/(1+m_ev)))
    return logLz

def log_LF(S, z):
    '''
    LF(logL_z, z, para)\n
    z is redshift.\n
    para is the parameter catalog of a particular type of galaxy. need to be a dictionary
    '''
    logn0 = para["logn0"]
    logL_star0 = para['logL*0']
    a = para['a']
    b = para['b']
    dz = 0.01
    logLz, logL0 = logL_Sz(S, z)
    dlogL_z0 = logL_z(logL0, dz)-logL_z(logL0, 0)
    dlogL_z = logL_z(logL0, z+dz)-logL_z(logL0, z)
    part1 = logn0 - np.log[10**(a*(logL0-logL_star0))+10**(b*(logL0-logL_star0))]
    part2 = np.log[dlogL_z0] - np.log[dlogL_z]
    logPhi = part1 + part2
    return logPhi

#count the number density
def Numbercounts_in_box(S, z, **others):
    ''' 
    we expect to know the number of galaxies in a little box located at a particular z and solid angle.\n
    These contribute the same flux S with different luminosities depended on z.\n
    input S, z, para and others. Others include Omega, deltaz, Deltaz and deltaS.\n
    returen number of galaxies in this box contributing flux S
    '''
    something = {'deltaz':0.01,'Deltaz':0.1,'deltaS':0.1,'Omega':1}#defualt parameters
    if others:
        something.update(others)
    deltaz = something['deltaz']
    Deltaz = something['Deltaz']
    deltaS = something['deltaS']
    Omega = something['Omega']
    H0 = 67.77
    c = 299792.458
    z_range = np.arange(z, Deltaz, deltaz)
    for z in z_range:
        logPhi_z = log_LF(S, z)
        dV = Omega*z**2*(c/H0)**3*deltaz#unit:Mpc^3
        dlogL = log_LF(S+deltaS, z+deltaz)-log_LF(S, z)
        N_z += 10**logPhi_z*dV*dlogL
    return N_z
    
    

In [1]:
'''
3.Cut the space into some small solid angles, each solid angle related to a small volume. 
Count the total number for this small volume. 
For each solid angle, give each halo a number of galaxies that is related to its halo mass, 
and with a total galaxies number counted before.
'''
def cut_space(halo, z, Theta, Phi, **others):
    ''' 
    cut the little box out\n
    size include DeltaTheta, DeltaPhi, Deltaz
    return a wanted box held halos
    '''
    size = {'DeltaTheta':1, 'DeltaPhi':1, "Deltaz": 0.1}
    if others:
        size.update(others)
    else:
        pass
    halo_box = {}
    for i in range(len(halo['#ID(1)'])):
        if (halo['Z'][i]>= z) and (halo['Z'][i]< z+size['Deltaz']) and (halo['Theta'][i]>= Theta) and (halo['Theta'][i]<Theta+size['DeltaTheta']) and (halo['Phi'][i]>=Phi) and (halo['Phi'][i]<size['DeltaPhi']):
            for key in halo:
                halo_box[key][i].append(halo[key][i])
        else:
            pass
    return halo_box

#simulate galaxies numbers in one little box
def galaxy_simulate(halo_box, z, S, para):
    ''' 
    return a new box with galaxies' numbers simulation
    '''
    number = Numbercounts_in_box(S, z, para)
    M_total = sum(halo_box['Mvir(4)'])
    halo_box['galaxy'] = []
    for i in range(len(halo_box['#ID(1)'])):
        n_i = round(number*halo_box['Mvir(4)'][i]/M_total)
        halo_box['galaxy'].append(n_i)
    return halo_box

#simulate in whole sky under a particular S
def all_sky_simulate(halo_dic, S, Deltaz = 0.1, DeltaS=0.1, size = 200, para):
    ''' 
    return a whole dictionary with numbers of galaxies in each halo
    '''
    #cut the space:
    Phi_range = np.arange(0, 2*np.pi, 1)
    Theta_range = np.arange(0,np.pi,1)
    z_max =  size/2 * H0 / c / 1000
    Z_range = np.arange(0,z_max, Deltaz)
    halo_with_galaxy = [0]
    for i in range(len(Phi_range)):
        phi = Phi_range[i]
        for j in range(len(Theta_range)):
            theta = Theta_range[j]
            for k in range(len(Z_range)):
                z = Z_range[k]
                halo_box = cut_space(halo_dic, z, theta, phi)
                halo_box = galaxy_simulate(halo_box, z, S, para )
                halo_with_galaxy += halo_box
    return halo_with_galaxy


#draw pictures
def plot_particular_S(halo_with_galaxy):
                   


    


SyntaxError: non-default argument follows default argument (1891232685.py, line 41)

# Step4: Add all luminocites in a total map

In [ ]:
''' 
1.Count all luminocity bin using method mentioned before.
    2.Make a big array, to hold total luminocity information. That is to say, add all arrays into a whole array, each with a particular luminocity.
    3.Return this big array
    4.Draw picture. You can see a huge amount of points, each shows to a galaxy, with a particular luminocity.
'''

def all_flux_count(halo_dic, S_max, S_min, para, DeltaS = 0.1):
    S_range = np.arange(S_min, S_max, DeltaS)
    all_flux_galaxy = {"flux":[], "halo_with_galaxy":[]}
    for S in S_range:
        all_flux_galaxy["flux"].append(S)
        halo_with_galaxy_S = all_sky_simulate(halo_dic, S, para)
        all_flux_galaxy["halo_with_galaxy"].append(halo_with_galaxy_S)
    return all_flux_galaxy




In [ ]:
#change a frequency, that every S transfer to another S, depend on the S-nu power law or others.
def logS_nu(nu2, logS1, alpha, nu1 = 1.4):
    sigma = 0.25
    beta = 0
    logS2 = (alpha - sigma**2*(1-beta)*np.ln(nu2/nu1))*np.log(nu2/nu1) + logS1
    return logS2

def change_frequency(nu2,all_flux_galaxy, para):
    alpha = para['alpha']
    for i in range(len(all_flux_galaxy["flux"])):
        logS1 = np.log(all_flux_count["flux"][i])
        all_flux_galaxy["flux"][i] = logS_nu(nu2, logS1, alpha)
    return all_flux_galaxy

